In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain -U
!pip install openai
!pip install pymysql
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 7.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from langchain import OpenAI, ConversationChain, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms import AzureOpenAI
from langchain.memory import ConversationEntityMemory
from langchain.memory import ConversationBufferMemory,ConversationBufferWindowMemory,ConversationSummaryMemory
from dotenv import load_dotenv
import time
import os
import openai

load_dotenv('/content/drive/MyDrive/vars.env')
os.environ["OPENAI_API_KEY"] = os.getenv('api_key')

In [ ]:
from urllib.parse import quote_plus
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain
user=os.getenv('user_name')
password=quote_plus(os.getenv('password'))
endpoint_url=os.getenv('endpoint_url')
database=os.getenv('database')
table_name=str(os.getenv('table_name'))

#database_uri
db_uri = f"mysql+pymysql://{user}:{password}@{endpoint_url}/{database}"

# make connection with sql server
db = SQLDatabase.from_uri(db_uri, include_tables=[table_name])
llm = OpenAI(temperature=0, verbose=True)

In [ ]:
memory = ConversationBufferWindowMemory(k=5)
db_chain = SQLDatabaseChain.from_llm(llm, db ,  memory=memory, verbose=True, top_k=5 )

In [ ]:

query="Tell me about the price of property located at 24 hilllbank "
db_chain.run(query)



> Entering new  chain...
Tell me about the price of property located at 24 hilllbank 
SQLQuery:SELECT current_price FROM last_listings_new WHERE addr_strt_name = 'HILLBANK' AND addr_streetnum = '24';
SQLResult: [(Decimal('192900.00000'),)]
Answer:The price of the property located at 24 Hillbank is $192,900.
> Finished chain.


'The price of the property located at 24 Hillbank is $192,900.'

In [ ]:
query="Are pets allowed on the property?"
db_chain.run(query)



> Entering new  chain...
Are pets allowed on the property?
SQLQuery:SELECT features FROM last_listings_new WHERE CURDATE() < listing_update_timestamp LIMIT 5;
SQLResult: []
Answer:Pets are not allowed on the property.
> Finished chain.


'Pets are not allowed on the property.'

## Query with Prompt Template and Few Shot examples

In [ ]:
_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run,
        then look at the results of the query and return the answer. Use the following format:

        Question: "Question here"
        SQLQuery: "SQL Query to run"
        SQLResult: "Result of the SQLQuery"
        Answer: "Final answer here"

        Question: what is the price of 11 CHALFONT RD TORONTO, ON M9W3R9
        SQLQuery: SELECT current_price FROM table_name WHERE match(unparsed_address) AGAINST('11 CHALFONT RD TORONTO, ON M9W3R9') AND addr_streetnum='11' and addr_cityname='TORONTO' and addr_pcode ='M9W3R9' and addr_strt_name = 'CHALFONT' LIMIT 5;
        SQLResult: [(Decimal('324900.00000')]
        Answer: The price of the property located at 11 CHALFONT RD TORONTO, ON M9W3R9 is $324900.00000.

        Only use the following tables:
        {table_info}

        Question: {input}"""

PROMPT = PromptTemplate(
            input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE)
db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True)
db_chain.run("what is the price of 50 DANJOHN CRES TORONTO, ON M1V3N4")



> Entering new  chain...
what is the price of 50 DANJOHN CRES TORONTO, ON M1V3N4
SQLQuery:SELECT current_price FROM last_listings_new WHERE match(unparsed_address) AGAINST('50 DANJOHN CRES TORONTO, ON M1V3N4') AND addr_streetnum='50' and addr_cityname='TORONTO' and addr_pcode ='M1V3N4' and addr_strt_name = 'DANJOHN' LIMIT 5;
SQLResult: [(Decimal('369900.00000'),)]
Answer:The price of the property located at 50 DANJOHN CRES TORONTO, ON M1V3N4 is $369900.00000.
> Finished chain.


'The price of the property located at 50 DANJOHN CRES TORONTO, ON M1V3N4 is $369900.00000.'

## Query with Context

In [ ]:
class Conversation:
    def __init__(self):
        self.question_answers = []

    def add_question_answer(self, question, answer):
        self.question_answers.append((question, answer))
        if len(self.question_answers) > 5:
            self.question_answers = self.question_answers[-5:]

    def get_last_question_answers(self):
        return self.question_answers



In [ ]:
conversation = Conversation()

In [ ]:
def make_chat_conversation(conversation_tuples):
    conversation_happened=''
    for tuple in conversation_tuples:
      Human, AI =tuple
      Human_prompt= "Human: " + Human
      AI_prompt= "AI: " + AI
      conversation_happened=conversation_happened+ Human_prompt +"\n" +AI_prompt
      # print(Human_prompt)
      # print(AI_prompt)
    return conversation_happened

def get_whole_conversation(question):
    conversation_happened = make_chat_conversation(conversation.get_last_question_answers())
    conversation_with_question = conversation_happened +"\n" + "Human: " + question

    response = db_chain.run(conversation_with_question)
    conversation.add_question_answer(question, response)
    last_question_answers = conversation.get_last_question_answers()
    return response


In [ ]:
question="Tell me about the property located at 81 Craighurst Ave, Toronto, ON, M4R1K1?"
get_whole_conversation(question)



> Entering new  chain...

Human: Tell me about the property located at 81 Craighurst Ave, Toronto, ON, M4R1K1?
SQLQuery:SELECT current_price, rooms_total, beds, baths, sqftbuilding, yearbuilt, prop_type, stories FROM last_listings_new WHERE match(unparsed_address) AGAINST('81 Craighurst Ave, Toronto, ON, M4R1K1') AND addr_streetnum='81' and addr_cityname='Toronto' and addr_pcode ='M4R1K1' and addr_strt_name = 'Craighurst' LIMIT 5;
SQLResult: [(Decimal('929900000.00000'), 8, 3, 2, None, 2023, 501, None)]
Answer:The property located at 81 Craighurst Ave, Toronto, ON, M4R1K1 has a current price of $929900000.00000, 8 rooms, 3 beds, 2 baths, no sqftbuilding, yearbuilt 2023, prop_type 501, and no stories.
> Finished chain.


'The property located at 81 Craighurst Ave, Toronto, ON, M4R1K1 has a current price of $929900000.00000, 8 rooms, 3 beds, 2 baths, no sqftbuilding, yearbuilt 2023, prop_type 501, and no stories.'

In [ ]:
question="How long has the property been on the market?"
get_whole_conversation(question)



> Entering new  chain...
Human: Tell me about the property located at 81 Craighurst Ave, Toronto, ON, M4R1K1?
AI: The property located at 81 Craighurst Ave, Toronto, ON, M4R1K1 has a current price of $929900000.00000, 8 rooms, 3 beds, 2 baths, no sqftbuilding, yearbuilt 2023, prop_type 501, and no stories.
Human: How long has the property been on the market?
SQLQuery:SELECT listing_update_timestamp FROM last_listings_new WHERE match(unparsed_address) AGAINST('81 Craighurst Ave, Toronto, ON, M4R1K1') AND addr_streetnum='81' and addr_cityname='Toronto' and addr_pcode ='M4R1K1' and addr_strt_name = 'Craighurst' LIMIT 5;
SQLResult: [(datetime.datetime(2017, 8, 10, 12, 7, 20),)]
Answer:The property located at 81 Craighurst Ave, Toronto, ON, M4R1K1 has been on the market since August 10, 2017.
> Finished chain.


'The property located at 81 Craighurst Ave, Toronto, ON, M4R1K1 has been on the market since August 10, 2017.'

In [ ]:
question="Is there a yard or outdoor space in this property?"
get_whole_conversation(question)



> Entering new  chain...
Human: Tell me about the property located at 81 Craighurst Ave, Toronto, ON, M4R1K1?
AI: The property located at 81 Craighurst Ave, Toronto, ON, M4R1K1 has a current price of $929900000.00000, 8 rooms, 3 beds, 2 baths, no sqftbuilding, yearbuilt 2023, prop_type 501, and no stories.Human: How long has the property been on the market?
AI: The property located at 81 Craighurst Ave, Toronto, ON, M4R1K1 has been on the market since August 10, 2017.
Human: Is there a yard or outdoor space in this property?
SQLQuery:SELECT features FROM last_listings_new WHERE match(unparsed_address) AGAINST('81 Craighurst Ave, Toronto, ON, M4R1K1') AND addr_streetnum='81' and addr_cityname='TORONTO' and addr_pcode ='M4R1K1' and addr_strt_name = 'CRAIGHURST' LIMIT 5;
SQLResult: [(None,)]
Answer:There is no yard or outdoor space in the property located at 81 Craighurst Ave, Toronto, ON, M4R1K1.
> Finished chain.


'There is no yard or outdoor space in the property located at 81 Craighurst Ave, Toronto, ON, M4R1K1.'